# SMARTER vs Vargoats
In this notebook we try to determine how many samples are in common between *Vargoats* and *SMARTER*. First of all, load *Vargoats* data table

In [ ]:
import pandas as pd

from src.features.smarterdb import global_connection, SampleGoat

In [ ]:
conn = global_connection()

In [ ]:
vargoats = pd.read_excel("VarGoats data access.xlsx", header=1)
vargoats.head()

In [ ]:
vargoats.info()

Ok, try to explore `ADAPTmap ID`:

In [ ]:
vargoats["ADAPTmap ID"].value_counts()

Well, about `897` vargoats animals don't have and adaptmap id. Get all the adaptmap ids:

In [ ]:
vargoats["ADAPTmap ID"]

I see that some animals have extra characters in their name. Try to normalize those samples:

In [ ]:
vargoats["ADAPTmap ID"] = vargoats["ADAPTmap ID"].apply(lambda name: name.split('*')[0])

Ok, get all samples from SMARTER database:

In [ ]:
samples = SampleGoat.objects.all()
samples.count()

Now read all data into dataframe:

In [ ]:
smarter = pd.read_json(samples.to_json())
smarter["dataset_id"] = smarter["dataset_id"].apply(lambda name: name['$oid'])
smarter.drop("_id", axis=1, inplace=True)
smarter.head()

In [ ]:
merged_datasets = vargoats.set_index("ADAPTmap ID").join(smarter.set_index("original_id"), lsuffix="vargoats", rsuffix="smarter", how="outer")
merged_datasets.head()
merged_datasets.info()

In [ ]:
merged_datasets.to_excel("merged_datasets.xlsx")

Is it possible that I have a vargoat original id in smarter but outside adaptmap? get all the non adaptmap samples from vargoats:

In [ ]:
original_ids = vargoats[vargoats["ADAPTmap ID"] == "not applicable"]["Original ID"]

In [ ]:
samples = SampleGoat.objects.filter(original_id__in=original_ids.to_list())
samples.count()

Vargoats has its own id representation